<a href="https://colab.research.google.com/github/aminayusif/PurchaseIQ/blob/main/PurchaseIQ_CustomerSegmentation_Using_K_Means_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction

This notebook performs customer segmentation using K-Means clustering on e-commerce consumer behavior data. It includes steps for data loading, exploration, preprocessing, clustering, cluster analysis, and anomaly detection using Isolation Forest. The goal is to identify distinct customer segments and unusual purchasing patterns to inform targeted marketing strategies.

Key Sections:

Data Loading and Exploration

Data Preprocessing

Customer Segmentation (K-Means Clustering)

Cluster Analysis and Interpretation

Anomaly Detection (Isolation Forest)

Marketing Recommendations based on Segments and Anomalies

### Data Loading and Exploration

#### Import common libaries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### Load dataset

In [2]:
df = pd.read_csv('/content/ECommerce_consumer behaviour.csv')


In [3]:
# Display the first few rows and info
display(df.head())
display(df.info())

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019501 entries, 0 to 2019500
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   department_id           int64  
 10  department              object 
 11  product_name            object 
dtypes: float64(1), int64(9), object(2)
memory usage: 184.9+ MB


None

Let's check for null values in the dataset

In [4]:
df.isnull().sum()

,0
order_id,0
user_id,0
order_number,0
order_dow,0
order_hour_of_day,0
days_since_prior_order,124342
product_id,0
add_to_cart_order,0
reordered,0
department_id,0
